# meta

> Metadata introspection for the NVIDIA monitor plugin used by cjm-ctl to generate the registration manifest

In [ ]:
#| default_exp meta

In [ ]:
#| export
import sys
from typing import Dict, Any

## get_plugin_metadata

Returns metadata required to register this plugin with the PluginManager. This function is called by `cjm-ctl` during the manifest generation phase.

The returned dictionary includes:

- **python_path**: Absolute path to the environment's Python interpreter
- **module/class**: Import path for dynamic loading by the Universal Worker
- **resources**: Hardware requirements (this plugin doesn't require GPU itself)

In [ ]:
#| export
def get_plugin_metadata() -> Dict[str, Any]:  # Plugin metadata for manifest generation
    """Return metadata required to register this plugin with the PluginManager."""
    return {
        "name": "cjm-system-monitor-nvidia",
        "version": "1.0.0",
        "type": "infrastructure",
        
        "module": "cjm_system_monitor_nvidia.plugin",
        "class": "NvidiaMonitorPlugin",
        
        # Absolute path to this environment's Python interpreter
        "python_path": sys.executable,
        
        # This plugin monitors GPUs but doesn't require GPU resources itself
        "resources": {
            "requires_gpu": False,
            "min_gpu_vram_mb": 0,
            "min_system_ram_mb": 128
        },
        
        "env_vars": {}
    }

### CLI Introspection

The `cjm-ctl install-all` command runs this function inside the plugin's conda environment:

```bash
conda run -n cjm-sys-mon-nvidia python -c \
  'from cjm_system_monitor_nvidia.meta import get_plugin_metadata; \
   import json; print(json.dumps(get_plugin_metadata(), indent=2))'
```

The output is written to `~/.cjm/plugins/cjm-system-monitor-nvidia.json`.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()